In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as smi
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statistics import mean
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, GRU, Dropout, Conv1D, MaxPooling1D, Flatten, RepeatVector
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from statsmodels.tsa.seasonal import seasonal_decompose
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from hyperopt import hp, Trials, fmin, tpe
import joblib
import warnings
warnings.simplefilter('ignore')

In [3]:
df=pd.read_csv(r'Niftymultifatorinput_with sensex and bank nifty data.csv')
df.head()

,Date,NIFTY,Crude Oil,India VIX,INRUSD,Gold,S & P 500,Sensex,NIFTY BANK
0,1/2/2009,3046.750000,46.340000,40.67,48.24,878.8,931.80,9958.219727,5195.10
1,1/5/2009,3121.449951,48.810001,38.60,48.23,857.2,927.45,10275.599610,5359.40
2,1/6/2009,3112.800049,48.580002,38.73,48.51,865.4,934.70,10335.929690,5395.30
3,1/7/2009,2920.399902,42.630001,44.36,48.55,841.1,906.65,9586.879883,4984.55
4,1/9/2009,2873.000000,41.700001,47.82,48.47,854.3,890.35,9406.469727,4906.70


In [4]:
feature_columns = ['NIFTY BANK', 'INRUSD', 'S & P 500']
target_column = ['NIFTY BANK']
features = df[feature_columns]
target = df[target_column].shift(-1)
target=target.dropna()

features =df[feature_columns].head(-1)
features.tail(5)

,NIFTY BANK,INRUSD,S & P 500
3668,47840.15,83.27,4746.75
3669,47491.85,83.13,4754.63
3670,47724.85,83.17,4774.75
3671,48282.20,83.18,4781.58
3672,48508.55,83.27,4783.35


In [5]:
target.tail(5)

,NIFTY BANK
3668,47491.85
3669,47724.85
3670,48282.20
3671,48508.55
3672,48292.25


In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)
scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))

n_steps = 5  # Number of timesteps to look back
X, y = [], []

for i in range(n_steps, len(scaled_features)):
    X.append(scaled_features[i-n_steps:i])
    y.append(scaled_target[i])

X = np.array(X)
y = np.array(y)

split=int(0.8*len(X))
X_train, y_train= X[:split], y[:split]
X_test, y_test= X[split:], y[split:]

In [7]:
#load saved LSTM 1 layer best model
model_lstm = load_model('model_ET_sp-usdinr_lstm1_0.9752.keras')

#Test the model with Outsample data for model Validation
model_lstm.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTM, predictions_LSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTM, predictions_LSTM)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
R2: 0.9672
MAPE: 0.0153
RMSE: 753.2874
RMSE_normalizeddata: 0.0167
MAE_normalizeddata: 0.0129


In [8]:
#load saved GRU 1 layer best model
model_gru = load_model('model_ET_sp-usdinr_gru1_0.9775.keras')

#Test the model with Outsample data for model Validation
model_gru.predict(X_test)
predictions_gru = model_gru.predict(X_test)
predictions_GRU = scaler.inverse_transform(predictions_gru)
y_test_actual_GRU = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRU, predictions_GRU)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRU, predictions_GRU)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRU, predictions_GRU)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_gru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_gru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
R2: 0.9716
MAPE: 0.0136
RMSE: 700.9691
RMSE_normalizeddata: 0.0155
MAE_normalizeddata: 0.0114


In [9]:
#load saved GRU-LSTM 1 layer best model
model_grulstm = load_model('model_ET_sp-usdinr_grulstm1_0.9797.keras')

#Test the model with Outsample data for model Validation
model_grulstm.predict(X_test)
predictions_grulstm = model_grulstm.predict(X_test)
predictions_GRULSTM = scaler.inverse_transform(predictions_grulstm)
y_test_actual_GRULSTM = scaler.inverse_transform(y_test.reshape(-1, 1))


# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MSE: {mse:.4f}")

rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_grulstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_grulstm)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
R2: 0.9729
MAPE: 0.0134
MSE: 468443.3527
RMSE: 684.4292
MAE_normalizeddata: 0.0113
RMSE of Normalized Data: 0.0152


In [10]:
#load saved LSTM-GRU 1 layer best model
model_lstmgru = load_model('model_ET_sp-usdinr_lstmgru1_0.9697.keras')
model_lstmgru.predict(X_test)

#Test the model with Outsample data for model Validation
predictions_lstmgru = model_lstmgru.predict(X_test)
predictions_LSTMGRU = scaler.inverse_transform(predictions_lstmgru)
y_test_actual_LSTMGRU = scaler.inverse_transform(y_test.reshape(-1, 1))

dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstmgru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_lstmgru)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R2: 0.9595
MAPE: 0.0169
MSE: 701024.8804
RMSE: 837.2723
MAE: 644.6672
MAE_normalizeddata: 0.0143
RMSE of Normalized Data: 0.0185


In [11]:
#load saved LSTM 2 layer best model
model_lstm = load_model('model_ET_sp-usdinr_lstm2_0.9725.keras')

#Test the model with Outsample data for model Validation
model_lstm.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTM, predictions_LSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTM, predictions_LSTM)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae=mean_absolute_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
R2: 0.9642
MAPE: 0.0164
RMSE: 786.2616
RMSE_normalizeddata: 0.0174
MAE: 633.1089
MAE_normalizeddata: 0.0140


In [12]:
#load saved GRU 2 layer best model
model_gru = load_model('model_ET_sp-usdinr_gru2_0.9626.keras')

#Test the model with Outsample data for model Validation
model_gru.predict(X_test)
predictions_gru = model_gru.predict(X_test)
predictions_GRU = scaler.inverse_transform(predictions_gru)
y_test_actual_GRU = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRU, predictions_GRU)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRU, predictions_GRU)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRU, predictions_GRU)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
mse_normalizeddata=mean_squared_error(y_test, predictions_gru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")
mae=mean_absolute_error(y_test_actual_GRU, predictions_GRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_gru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
R2: 0.9540
MAPE: 0.0186
RMSE: 891.7981
RMSE_normalizeddata: 0.0197
MAE: 707.2846
MAE_normalizeddata: 0.0157


In [13]:
#load saved GRU-LSTM 2 layer best model
model_grulstm = load_model('model_ET_sp-usdinr_grulstm2_0.9594.keras')

#Test the model with Outsample data for model Validation
model_grulstm.predict(X_test)
predictions_grulstm = model_grulstm.predict(X_test)
predictions_GRULSTM = scaler.inverse_transform(predictions_grulstm)
y_test_actual_GRULSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_grulstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_grulstm)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
R2: 0.9441
MAPE: 0.0201
MSE: 966236.6857
RMSE: 982.9734
MAE: 771.5242
MAE_normalizeddata: 0.0171
RMSE of Normalized Data: 0.0218


In [14]:
#load saved LSTM-GRU 2 layer best model
model_lstmgru = load_model('model_ET_sp-usdinr_lstmgru2_0.9512.keras')

#Test the model with Outsample data for model Validation
model_lstmgru.predict(X_test)
predictions_lstmgru = model_lstmgru.predict(X_test)
predictions_LSTMGRU = scaler.inverse_transform(predictions_lstmgru)
y_test_actual_LSTMGRU = scaler.inverse_transform(y_test.reshape(-1, 1))

dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstmgru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_lstmgru)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R2: 0.9383
MAPE: 0.0211
MSE: 1067372.6314
RMSE: 1033.1373
MAE: 810.7365
MAE_normalizeddata: 0.0179
RMSE of Normalized Data: 0.0229


In [15]:
#load saved LSTM 3 layer best model
model_lstm = load_model('model_ET_sp-usdinr_lstm3_0.9501.keras')

#Test the model with Outsample data for model Validation
model_lstm.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTM, predictions_LSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTM, predictions_LSTM)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae=mean_absolute_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
R2: 0.9390
MAPE: 0.0215
RMSE: 1026.9990
RMSE_normalizeddata: 0.0227
MAE: 818.3474
MAE_normalizeddata: 0.0181


In [16]:
#load saved GRU 3 layer best model
model_gru = load_model('model_ET_sp-usdinr_gru3_0.9745.keras')

#Test the model with Outsample data for model Validation
model_gru.predict(X_test)
predictions_gru = model_gru.predict(X_test)
predictions_GRU = scaler.inverse_transform(predictions_gru)
y_test_actual_GRU = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRU, predictions_GRU)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRU, predictions_GRU)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRU, predictions_GRU)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
mse_normalizeddata=mean_squared_error(y_test, predictions_gru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")
mae=mean_absolute_error(y_test_actual_GRU, predictions_GRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_gru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
R2: 0.9673
MAPE: 0.0153
RMSE: 752.2934
RMSE_normalizeddata: 0.0167
MAE: 584.0185
MAE_normalizeddata: 0.0129


In [19]:
#load saved GRU-LSTM 3 layer best model
model_grulstm = load_model('model_ET_sp-usdinr_grulstm3_0.9396.keras')

#Test the model with Outsample data for model Validation
model_grulstm.predict(X_test)
predictions_grulstm = model_grulstm.predict(X_test)
predictions_GRULSTM = scaler.inverse_transform(predictions_grulstm)
y_test_actual_GRULSTM = scaler.inverse_transform(y_test.reshape(-1, 1))
dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
mae=mean_absolute_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_grulstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_grulstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE of Normalized Data: {rmse_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
R2: 0.9341
MAPE: 0.0214
MSE: 1138739.0531
RMSE: 1067.1172
MAE: 821.0607
MAE_normalizeddata: 0.0182
RMSE of Normalized Data: 0.0236


In [20]:
#load saved LSTM-GRU 3 layer best model
model_lstmgru = load_model('model_ET_sp-usdinr_lstmgru3_0.9420.keras')

#Test the model with Outsample data for model Validation
model_lstmgru.predict(X_test)
predictions_lstmgru = model_lstmgru.predict(X_test)
predictions_LSTMGRU = scaler.inverse_transform(predictions_lstmgru)
y_test_actual_LSTMGRU = scaler.inverse_transform(y_test.reshape(-1, 1))
dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstmgru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstmgru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE of Normalized Data: {rmse_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
R2: 0.9258
MAPE: 0.0232
MSE: 1282447.1287
RMSE: 1132.4518
MAE: 899.6542
MAE_normalizeddata: 0.0199
RMSE of Normalized Data: 0.0251
